In [6]:
%pip install requests
%pip install pandas
%pip install beautifulsoup4
%pip install boto3
%pip install logging

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\SSAFY\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\SSAFY\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\SSAFY\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\SSAFY\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/96.0 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/96.0 kB ? eta -:--:--
     -------------------------------------- - 92.2/96.0 kB 1.1 MB/s eta 0:00:01
     -------------------------------------- 96.0/96.0 kB 912.6 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for logging: filename=logging-0.4.9.6-py3-none-any.whl size=29527 sha256=d83497bdcc1218fd4dc9bc0c7b56e3f4ab849aebc7396ad4cfdb46c3d9557cc5
  Stored in directory: c:\users\ssafy\appdata\local\packages\pythonsoftwarefoundation.python.3.11_qbz5n2kfra8p0\localcache\local\pip\cache\wheels\30\80\e6\0a01576c333eb17cb6f3ad46889726bbb140ff661beae5ad2a
Succe


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\SSAFY\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# 중앙일보

## 🎈 테스트

### **상단 10건을 조회하여 한 파일에 저장**

In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import logging

logging.basicConfig(level='DEBUG')

joongang_news_df = pd.DataFrame(columns=['headline', 'time', 'content', 'reporter'])
idx = 0
header = {'User-Agent' : ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                          '(KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'),}

# 스크랩 함수
def scrape_article(article_url):
    response = requests.get(article_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    try:
        category = soup.find('a', { 'class': 'title' }).get_text(strip=True)
        if category != '정치': # 정치가 아니면 Skip
            logging.info(f"Skipping article {article_url}: not a Politics article.")
            return None
    except AttributeError:
        logging.error(f"Skipping article {article_url}: category not found.")
        return None

    try:
        headline = soup.find('h1', { 'class': 'headline' }).get_text(strip=True)
    except AttributeError:
        headline = 'N/A'

    try:
        date = soup.find('p', { 'class': 'date' })
        time = date.find('time').get('datetime')
    except AttributeError:
        time = 'N/A'

    try:
        content = soup.find('div', { 'class': 'article_body' }).get_text(strip=True)
    except AttributeError:
        content = 'N/A'

    try:
        byline = soup.find('div', { 'class': 'byline'})
        reporter = byline.find('a').get_text(strip=True).split('\n')[0]
    except AttributeError:
        reporter = 'N/A'

    return headline, time, content, reporter

# 스크랩 이후 중복 데이터 제거 후처리 함수
def remove_duplicates_by_time(df):
    # Remove rows where the 'time' field is the same, keeping the first occurrence
    df = df.drop_duplicates(subset='time', keep='first').reset_index(drop=True)
    return df

# Base URL
base_url = "https://www.joongang.co.kr/politics"
response = requests.get(base_url)
soup = BeautifulSoup(response.text, 'html.parser')

cards = soup.find_all('li', class_='card')
for card in cards:
    article_link = card.find('a', href=True)['href']
    logging.debug(f"Scraping article: {article_link}")

    # 1차 결과 검증
    result = scrape_article(article_link)

    if result is None:
        continue

    headline, time, content, reporter = result

    # 2차 결과 검증
    if 'N/A' in [headline, time, content, reporter]:
        logging.info(f"Skipping article because {[headline, time, content, reporter]}: missing required fields.")
        continue

    joongang_news_df.loc[idx] = [headline, time, content, reporter]
    idx += 1

# 최신순 정렬
# 1. time의 데이터를 datetime 형식으로 변경
joongang_news_df['time'] = pd.to_datetime(joongang_news_df['time'])

# 2. 시간 기준 내림차순 정렬
joongang_news_df = joongang_news_df.sort_values(by='time', ascending=False)

# 3. 시간 기준으로 중복 제거
joongang_news_df = remove_duplicates_by_time(joongang_news_df)

# 4. 다시 time 형식 변경
joongang_news_df['time'] = joongang_news_df['time'].dt.strftime('%Y-%m-%dT%H:%M:%S%z')

joongang_news_df.to_json('./joongang-articles/joongang_articles.json', orient='records', lines=True, force_ascii=False)

Scraping article: https://www.joongang.co.kr/article/25278499
Skipping article https://www.joongang.co.kr/article/25278499: not a Politics article.
Scraping article: https://www.joongang.co.kr/article/25278695
Skipping article https://www.joongang.co.kr/article/25278695: not a Politics article.
Scraping article: https://www.joongang.co.kr/article/25278704
Skipping article https://www.joongang.co.kr/article/25278704: not a Politics article.
Scraping article: https://www.joongang.co.kr/article/25278716
Skipping article https://www.joongang.co.kr/article/25278716: not a Politics article.
Scraping article: https://www.joongang.co.kr/article/25278507
Skipping article https://www.joongang.co.kr/article/25278507: not a Politics article.
Scraping article: https://www.joongang.co.kr/article/25278704
Skipping article https://www.joongang.co.kr/article/25278704: not a Politics article.
Scraping article: https://www.joongang.co.kr/article/25278705
Scraping article: https://www.joongang.co.kr/artic

## 🎈 테스트

### 1건씩 조회하여 단일 기사별 파일 생성
---

1. 파일 type: json

2. 파일명 형식: uuid

3. 필드

    - headline
    
    - time -> '%Y-%m-%dT%H:%M:%S%z'
    
    - content

    - reporter

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import uuid
import os

# 예외 처리
output_dir = './joongang-articles'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

header = {'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                         '(KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'),}

# 스크랩 함수
def scrape_article(article_url):
    response = requests.get(article_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    try:
        category = soup.find('a', {'class': 'title'}).get_text(strip=True)
        if category != '정치':  # 정치가 아니면 패스
            logging.info(f"Skipping article {article_url}: not a Politics article.")
            return None
    except AttributeError:
        logging.error(f"Skipping article {article_url}: category not found.")
        return None

    try:
        headline = soup.find('h1', {'class': 'headline'}).get_text(strip=True)
    except AttributeError:
        headline = 'N/A'

    try:
        date = soup.find('p', {'class': 'date'})
        time = date.find('time').get('datetime')
    except AttributeError:
        time = 'N/A'

    try:
        content = soup.find('div', {'class': 'article_body'}).get_text(strip=True)
    except AttributeError:
        content = 'N/A'

    try:
        byline = soup.find('div', {'class': 'byline'})
        reporter = byline.find('a').get_text(strip=True).split('\n')[0]
    except AttributeError:
        reporter = 'N/A'

    return {'headline': headline, 'time': time, 'content': content, 'reporter': reporter}

# 파일 생성
def save_article_as_json(article_data):
    file_name = str(uuid.uuid4()) + '.json'
    file_path = os.path.join(output_dir, file_name)
    
    # 단일 dataframe 생성
    single_article_df = pd.DataFrame([article_data])
    
    # json으로 변환
    single_article_df.to_json(file_path, orient='records', lines=True, force_ascii=False)
    logging.info(f"Article saved to {file_path}")

# 새로운 기사 스크랩
def check_for_new_articles():
    base_url = "https://www.joongang.co.kr/politics"
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    cards = soup.find_all('li', class_='card')
    for card in cards:
        article_link = card.find('a', href=True)['href']
        logging.info(f"Scraping article: {article_link}")

        # Scrape the article
        result = scrape_article(article_link)

        if result is None:
            continue

        save_article_as_json(result)

Scraping article: https://www.joongang.co.kr/article/25278499
Skipping article https://www.joongang.co.kr/article/25278499: not a Politics article.
Scraping article: https://www.joongang.co.kr/article/25278695
Skipping article https://www.joongang.co.kr/article/25278695: not a Politics article.
Scraping article: https://www.joongang.co.kr/article/25278704
Skipping article https://www.joongang.co.kr/article/25278704: not a Politics article.
Scraping article: https://www.joongang.co.kr/article/25278507
Skipping article https://www.joongang.co.kr/article/25278507: not a Politics article.
Scraping article: https://www.joongang.co.kr/article/25278788
Skipping article https://www.joongang.co.kr/article/25278788: not a Politics article.
Scraping article: https://www.joongang.co.kr/article/25278704
Skipping article https://www.joongang.co.kr/article/25278704: not a Politics article.
Scraping article: https://www.joongang.co.kr/article/25278705
Article saved to ./joongang-articles\7256080b-dda7

## 🎈 테스트

### S3 업로드

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import uuid
import boto3
import io

header = {'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                         '(KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'),}

session = boto3.Session(
    aws_access_key_id='deleteKey',
    aws_secret_access_key='deleteAccessKey',
    region_name='ap-northeast-2'
)

s3 = session.resource('s3')
bucket_name = 'newsseug-bucket'

def scrape_article(article_url):
    response = requests.get(article_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    try:
        category = soup.find('a', {'class': 'title'}).get_text(strip=True)
        
        # 카테고리가 아닌 글은 패스
        if category != '정치':
            logging.info(f"{article_url} 는 해당 카테고리가 아니어서 생략합니다. ")
            return None
    except AttributeError:
        logging.error(f"{article_url} 는 찾을 수 없습니다.")
        return None

    # 헤드라인 스크랩
    try:
        headline = soup.find('h1', {'class': 'headline'}).get_text(strip=True)
    except AttributeError:
        headline = 'N/A'

    # 생성일 스크랩
    try:
        date = soup.find('p', {'class': 'date'})
        time = date.find('time').get('datetime')
    except AttributeError:
        time = 'N/A'

    # 본문 스크랩
    try:
        content = soup.find('div', {'class': 'article_body'}).get_text(strip=True)
    except AttributeError:
        content = 'N/A'

    # 기자 이름 스크랩
    try:
        byline = soup.find('div', {'class': 'byline'})
        reporter = byline.find('a').get_text(strip=True).split('\n')[0]
    except AttributeError:
        reporter = 'N/A'

    return {'headline': headline, 'time': time, 'content': content, 'reporter': reporter}

def save_article_to_s3(article_data):
    file_name = str(uuid.uuid4()) + '.json'

    article_df = pd.DataFrame([article_data])

    buffer = io.StringIO()
    article_df.to_json(buffer, orient='records', lines=True, force_ascii=False)

    buffer.seek(0)
    
    try:
        # s3.put_object(Bucket=bucket_name, Key=f'content/{file_name}', Body=buffer.getvalue())
        s3.Bucket(bucket_name).put_object(Key=f'content/{file_name}', Body=buffer.getvalue())
        logging.debug(f"{file_name} 을 {bucket_name} 에 업로드 하였습니다.")
    except Exception as e:
        logging.error(f"Error uploading {file_name}: {e}")
        e

def check_for_new_articles():
    base_url = "https://www.joongang.co.kr/politics"
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    cards = soup.find_all('li', class_='card')
    for card in cards:
        article_link = card.find('a', href=True)['href']
        logging.info(f"Scraping article: {article_link}")

        result = scrape_article(article_link)

        if result is None:
            continue

        save_article_to_s3(result)

check_for_new_articles()


Scraping article: https://www.joongang.co.kr/article/25279089
https://www.joongang.co.kr/article/25279089 는 해당 카테고리가 아니어서 생략합니다. 
Scraping article: https://www.joongang.co.kr/article/25279569
https://www.joongang.co.kr/article/25279569 는 해당 카테고리가 아니어서 생략합니다. 
Scraping article: https://www.joongang.co.kr/article/25279444
https://www.joongang.co.kr/article/25279444 는 해당 카테고리가 아니어서 생략합니다. 
Scraping article: https://www.joongang.co.kr/article/25279621
f7e7629d-3749-4fe6-80ed-db8631746e6c.json 을 newsseug-bucket 에 업로드 하였습니다.
Scraping article: https://www.joongang.co.kr/article/25279565
https://www.joongang.co.kr/article/25279565 는 해당 카테고리가 아니어서 생략합니다. 
Scraping article: https://www.joongang.co.kr/article/25279621
27a37ea5-86a2-4efb-ac7d-3d8e3754a2ce.json 을 newsseug-bucket 에 업로드 하였습니다.
Scraping article: https://www.joongang.co.kr/article/25279352
https://www.joongang.co.kr/article/25279352 는 해당 카테고리가 아니어서 생략합니다. 
Scraping article: https://www.joongang.co.kr/article/25279599
1bdfd1e1-fbd0-4ef5

# 중앙일보 리팩터링

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import logging
from urllib.parse import urlparse

def lambda_handler(event, context):
    
    return {
        'statusCode': 200,
        'body': 'joongang-news-crawler from Lambda is do!'
    }

logging.basicConfig(level='DEBUG')

header = {'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                         '(KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'),}

categories = {
    '정치': 'https://www.joongang.co.kr/politics',
    '경제': 'https://www.joongang.co.kr/money',
    '국제': 'https://www.joongang.co.kr/world',
    '사회_사건': 'https://www.joongang.co.kr/society/accident', # 앞 분기 기준
    '경제_과학': 'https://www.joongang.co.kr/money/science', # 앞 분기 기준
    '사회': 'https://www.joongang.co.kr/society',
    '스포츠': 'https://www.joongang.co.kr/sports',
}


result_categories = {
    '정치': 'politics',
    '경제': 'economy',
    '국제': 'world',
    '사회_사건': 'accident',
    '경제_과학': 'science',
    '사회': 'society',
    '스포츠': 'sports',
}

def scrap_article(article_url, category):
    response = requests.get(article_url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    try:

        page_category = soup.find('a', {'class': 'title'}).get_text(strip=True)
        logging.info(f"page_category: {page_category}")
        
        logging.info(f"category: {category}")

        category_parts = category.split('_')
        
        # 카테고리가 아닌 글은 패스
        if page_category not in category_parts:
            logging.info(f"{article_url} 는 해당 카테고리가 아니어서 생략합니다. ")
            return None
    except AttributeError:
        logging.error(f"{article_url} 는 찾을 수 없습니다.")
        return None

    # 헤드라인 스크랩
    try:
        title = soup.find('h1', {'class': 'headline'}).get_text(strip=True)
    except AttributeError:
        title = 'N/A'

    # 생성일 스크랩
    try:
        date = soup.find('p', {'class': 'date'})
        time = date.find('time').get('datetime')
    except AttributeError:
        time = 'N/A'

    # 본문 스크랩
    try:
        content = soup.find('div', {'class': 'article_body'}).get_text(strip=True)
    except AttributeError:
        content = 'N/A'

    result_category = result_categories.get(category, 'unknown')

    return {'title': title, 'time': time, 'content': content, 'source_url': article_url, 'category': result_category}

def check_for_new_articles(base_url, category):
    response = requests.get(base_url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    article_count = 0
    cards = soup.find_all('li', class_='card')
    cards.reverse()
    for card in cards:
        if article_count >= 3:
            break

        link_tag = card.find('a', href=True)
        if link_tag and 'href' in link_tag.attrs:
            article_link = link_tag['href']
        else:
            logging.warning("article_link is None or missing 'href'.")
            continue
    
        logging.debug(f"Scraping article: {article_link}")

        result = scrap_article(article_link, category)

        if result is None:
            continue

        print(result)
        article_count += 1

def scrap_all_categories():
    
    for category, base_url in categories.items():
        logging.info(f"category (base_url): {category} ({base_url})")
        check_for_new_articles(base_url, category)
        time.sleep(2)

scrap_all_categories()

# 동아일보

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import logging
from urllib.parse import urlparse

def lambda_handler(event, context):
    
    return {
        'statusCode': 200,
        'body': 'joongang-news-crawler from Lambda is do!'
    }

logging.basicConfig(level='DEBUG')

header = {'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                         '(KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'),}

categories = {
    '정치': 'https://www.donga.com/news/Politics',
    '경제': 'https://www.donga.com/news/Economy',
    '국제': 'https://www.donga.com/news/Inter',
    '사회_사건': 'https://www.donga.com/news/Society/Event', # 앞 분기 기준
    '경제_과학': 'https://www.donga.com/news/It/List', # 앞 분기 기준
    '사회': 'https://www.donga.com/news/Society',
    '스포츠': 'https://www.donga.com/news/Sports',
}

result_categories = {
    '정치': 'politics',
    '경제': 'economy',
    '국제': 'world',
    '사회_사건': 'accident',
    '경제_과학': 'science',
    '사회': 'society',
    '스포츠': 'sports',
}

def scrap_article(article_url, category):
    response = requests.get(article_url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    try:
        page_category = soup.find('ol', class_='breadcrumb').find('li', class_='breadcrumb_item').find('a').get_text(strip=True)
        logging.info(f"page_category: {page_category}")
        
        logging.info(f"category: {category}")

        category_parts = category.split('_')
        
        # 카테고리가 아닌 글은 패스
        if page_category not in category_parts:
            logging.info(f"{article_url} 는 해당 카테고리가 아니어서 생략합니다. ")
            return None
    except AttributeError:
        logging.error(f"{article_url} 는 찾을 수 없습니다.")
        return None

    # 헤드라인 스크랩
    try:
        title = soup.find('section', {'class': 'head_group'}).find('h1').get_text(strip=True)
    except AttributeError:
        title = 'N/A'
        
    try:
        date_button = soup.find('button', {'aria-controls': 'dateInfo'})
        time_span = date_button.find('span', {'aria-hidden': 'true'})
        time = time_span.get_text(strip=True)  # "2024-09-30 21:44" 형태
        time = time.replace(" ", "T") + ":00+09:00"  # ISO 8601 형식으로 변환 (예: 2024-09-30T21:44:00+09:00)
    except AttributeError:
        time = 'N/A'

    # 본문 스크랩
    try:
        # 광고, 이미지 등 제외하고 텍스트만 추출
        content_section = soup.find('section', class_='news_view')
        for tag in content_section(['figure', 'script', 'div', 'ul']):  # 불필요한 태그 제거
            tag.decompose()
        content = content_section.get_text(separator="\n", strip=True)
    except AttributeError:
        content = 'N/A'

    result_category = result_categories.get(category, 'unknown')

    return {'title': title, 'time': time, 'content': content, 'source_url': article_url, 'category': result_category}

def check_for_new_articles(base_url, category):
    response = requests.get(base_url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    article_count = 0
    cards = soup.find_all('article', class_='news_card')
    cards.reverse()
    for card in cards:
        if article_count >= 3:
            break

        link_tag = card.find('a', href=True)
        if link_tag and 'href' in link_tag.attrs:
            article_link = link_tag['href']
        else:
            logging.warning("article_link is None or missing 'href'.")
            continue
    
        logging.debug(f"Scraping article: {article_link}")

        result = scrap_article(article_link, category)

        if result is None:
            continue

        print(result)
        article_count += 1

def scrap_all_categories():
    
    for category, base_url in categories.items():
        logging.info(f"category (base_url): {category} ({base_url})")
        check_for_new_articles(base_url, category)
        time.sleep(2)

scrap_all_categories()

INFO:root:category (base_url): 정치 (https://www.donga.com/news/Politics)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Politics HTTP/1.1" 200 None
DEBUG:root:Scraping article: https://www.donga.com/news/Society/article/all/20240930/130131088/1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Society/article/all/20240930/130131088/1 HTTP/1.1" 200 None
INFO:root:page_category: 사회
INFO:root:category: 정치
INFO:root:https://www.donga.com/news/Society/article/all/20240930/130131088/1 는 해당 카테고리가 아니어서 생략합니다. 
DEBUG:root:Scraping article: https://www.donga.com/news/Politics/article/all/20240930/130130608/1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Politics/article/all/20240930/130130608/1 HTTP/1

{'title': '친한 “대형 악재, 金사과해야” 野 “10월 김건희 스모킹건 찾을것”', 'time': '2024-09-30T21:17:00+09:00', 'content': '국회로 돌아올 ‘김건희 여사 특검법’ 재표결을 앞두고 여권 내부의 고심이 깊어지고 있다. 국민의힘은 친윤(친윤석열)계인 원내지도부를 중심으로 이탈표 방지를 위한 표 단속에 나섰다. 하지만 여당 내부에선 친한(친한동훈)계를 중심으로 “특검법 자체가 대형 악재”라며 “특검법이 부결되더라도 김 여사가 사과해야 한다”는 목소리가 커지고 있다. 대통령실은 윤석열 대통령의 ‘김건희 특검법’ 거부권 행사를 기정사실화하면서도 거부권 행사에 대한 비판 여론이 적지 않은 만큼 4일 시한까지 염두에 두고 윤 대통령 재가 시점을 고민 중인 것으로 알려졌다.\n더불어민주당은 30일 윤 대통령과 김 여사를 겨냥한 ‘국정농단 제보센터’를 띄우는 등 김 여사 의혹을 파헤치기 위한 총력전에 돌입하며 여당 내 이탈표를 압박하고 나섰다. 민주당은 이달 7일 시작되는 국정감사에서 김 여사 의혹에 대한 스모킹건이 확인될 경우 탄핵 국면이 본격화될 수 있다고 보고 있다. 일부 의원들이 특검법 수용 촉구 천막 농성에 돌입한 민주당은 윤 대통령이 재의요구권(거부권)을 행사하면 이르면 4일, 늦어도 토요일인 5일 국회 본회의를 열어 재표결하겠다는 계획이다. 김 여사 총선 개입 의혹 관련 공직선거법상 공소시효가 만료되는 이달 10일 전까지 관련 절차가 마무리돼야 한다는 것이다. 이언주 최고위원은 “원리상의 한계를 넘어선 거부권 행사는 위헌이기 때문에 그 자체가 탄핵 소추 사유”라고 했다.\n● 與 내부 “특검법 대형 악재, 金 사과해야”\n국민의힘 추경호 원내대표는 이날 오전 회의에서 “재의하게 되면 국민의힘 의원들은 하나로 똘똘 뭉쳐 폐기시킬 것”이라고 말했다. 한 친윤 의원도 “당연히 부결될 것”이라고 했다. 원내 지도부 소속 한 의원은 “지금 민주당이 발의하는 특검법들은 이재명 대표 방탄용이고 정쟁용이란 것을 의원들이 다 

INFO:root:page_category: 정치
INFO:root:category: 정치
DEBUG:root:Scraping article: https://www.donga.com/news/Politics/article/all/20240930/130132806/2
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443


{'title': '韓-슬로바키아 ‘전략적 동반자관계’ 수립…원전 등 에너지 협력 강화키로', 'time': '2024-09-30T19:24:00+09:00', 'content': '윤석열 대통령이 30일 공식 방한한 로베르트 피초 슬로바키아 총리와 회담을 갖고 한-슬로바키아 간 ‘전략적 동반자관계’를 수립하기로 하고 원자력 등 에너지와 국방·방산 등 분야에서 협력을 강화하기로 했다.\n윤 대통령은 이날 정상회담에서 “오늘 회담이 양국 간 교역과 투자를 비롯해 에너지, 공급망 등 포괄적인 분야에서 호혜적 협력을 새로운 차원으로 끌어올리는 계기가 되기를 기대한다”고 밝혔다. 피초 총리도 “원자력 분야 협력에 있어 긍정적인 반응을 보여주셔서 감사하다”며 “원자력을 포함해서 무역, 경제 투자 분야의 협력을 논의하고자 한다”고 밝혔다.\n피초 총리가 방한한 건 첫 재임 당시인 2007년에 이어 두 번째다. 올 5월 지지자들을 만나다 피격을 당했던 피초 총리는 윤 대통령에게 “당시 저의 빠른 쾌유를 빌어주신 것에 대해 깊은 감사의 말씀을 드린다”고도 했다.\n회담 결과 양국은 전략적 동반자관계 수립과 함께 교역·투자, 에너지, 국방·방산 등 핵심 분야를 중심으로 양국 관계를 발전시켜 나가기로 했다. 슬로바키아가 아시아국가와 전략적 동반자관계를 맺은 건 한국이 처음이고 한국도 이로써 슬로바키아, 체코, 폴란드, 헝가리 등 비세그라드그룹 4개국 전체와 전략적 동반자관계를 수립하게 됐다.\n양국은 이날 무역투자촉진프레임워크(TIPF) 수립, 포괄적 에너지 협력 등 양해각서(MOU)도 체결했다. 슬로바키아가 올해 5월 자국 원전 단지에 1200㎿ 원전을 신규 건설하는 계획을 승인했던 만큼 양국 간 에너지 협력 강화가 원전 수주로 이어질 수도 있을 것이라는 관측도 나온다.', 'source_url': 'https://www.donga.com/news/Politics/article/all/20240930/130131831/1', 'category': 'politics'}


DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Politics/article/all/20240930/130132806/2 HTTP/1.1" 200 None
INFO:root:page_category: 정치
INFO:root:category: 정치


...

{'title': '축구협회, 언제쯤 정상화될까…이틀 후 국정감사로 ‘시끌’', 'time': '2024-09-30T08:17:00+09:00', 'content': 'A매치 임박했지만 어수선한 분위기 계속\n내달 2일 문체부 감사 중간 결과 발표 예정\n정 회장은 국회 국정감사 증인 출석도 앞둬\nA매치가 임박했지만 대한축구협회를 둘러싼 잡음으로 어수선한 분위기가 이어지고 있다.\n홍명보 감독이 이끄는 축구 국가대표팀은 내달 2026 국제축구연맹(FIFA) 북중미 월드컵‘ 아시아 3차 예선 B조 3차전에서 요르단(10월10일), 4차전에서 이라크(10월15일)를 상대한다.\n한국 축구는 11회 연속 월드컵 본선 진출이라는 대업에 도전하지만, 축구협회의 여러 가지 논란과 의혹으로 응원받지 못하고 있다.\n30일 오전 서울시 종로구의 축구회관에서 예정된 대표팀 명단 발표도 여느 때보다 주목도가 떨어진 분위기다.\n홍 감독은 부임 과정이 공정하지 못했다는 이유로 비판받고, 정몽규 축구협회장은 4선 연임 도전 가능성과 승부 조작범 사면 등으로 압박받고 있다.\n지난 25일 축구협회 주요 인사들은 국회 문화체육관광위원회 현안 질의에 증인으로 출석했지만, 범국민적 분노를 해소하지 못했다.\n이날 홍 감독, 정 회장과 함께 출석한 이임생 기술총괄이사는 한 전력강화위원과 나눈 메시지에서 사후 회유 의혹을 받고 사퇴의 뜻을 밝히기까지 했다.\n당시 더불어민주당 양문석 의원은 “(전력강화위원회의) 11차 회의도 불법이고, 이 이사가 위임받은 부분도 불법이고, 그 불법의 토대 위에서 서류 제출도 안 하고, 사전 면접도 안 하고, 실제 대면 면접을 한, 설득을 한 홍 감독이 선임됐다”며 꼬집었다.\n국민의힘 배현진 의원은 “홍 감독 선임 과정이 대단히 억지스럽고, 그 과정 중에 절차적인 정당성에 문제가 발생했다”며 “특정한 학교의 학연에 근거한 카르텔이 있는 것 아니냐는 소문이 돌 정도”였다며 증인으로 나선 축구협회 인사들을 비판했다.\n축구협회를 향한 압박은 더욱 거세질 전망이

INFO:root:page_category: 스포츠
INFO:root:category: 스포츠
DEBUG:root:Scraping article: https://www.donga.com/news/Sports/article/all/20240930/130128889/1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Sports/article/all/20240930/130128889/1 HTTP/1.1" 200 None


{'title': '부상으로 맨유전 결장한 손흥민…토트넘 감독 “어떻게 될지 지켜보자”', 'time': '2024-09-30T09:17:00+09:00', 'content': '허벅지 부상 손흥민, 30일 맨유전서 시즌 첫 결장\n토트넘, 10월4일 헝가리 페렌츠바로시 원정 경기\n엔제 포스테코글루 토트넘 홋스퍼 감독이 허벅지 부상 때문에 올 시즌 처음으로 결장한 손흥민의 상태에 대해 조심스러운 태도를 보였다.\n토트넘은 30일(이하 한국시간) 영국 맨체스터의 올드 트래퍼드에서 열린 맨체스터 유나이티드(이하 맨유)와의 2024-25 잉글랜드 프리미어리그 6라운드에서 3-0으로 승리했다.\n지난 15일 아스널과의 홈경기 패배 후 모든 대회를 통틀어 4경기 연속 승리다.\n경기 후 포스테코글루 감독은 “토트넘은 홈과 원정을 가리지 않고 좋은 축구를 선보이고 있다. 아직 많은 부분에서 개선해야 하지만 좋은 점은 계속 좋아지고 있다”고 밝혔다.\n이어 이날 결장한 손흥민에 대한 질문에 “아마 손흥민은 승리를 기뻐하며 뛰어다닐 것”이라고 웃은 뒤 “두고 보자. 런던으로 돌아가 손흥민의 몸 상태를 지켜본 뒤 다음 경기 출전을 결정할 것이다. 어떻게 될지 지켜보자”고 말했다.\n손흥민은 지난 27일 카라바크(아제르바이잔)와 유럽축구연맹(UEFA) 유로파리그(UEL) 1차전에서 후반 26분 허벅지 통증을 호소한 뒤 교체로 물러났다.\n포스테코글루 감독은 손흥민의 훈련 때 몸 상태를 지켜본 뒤 맨유전 출전이 힘들다고 판단, 명단에서 제외했다. 올 시즌 첫 결장이다.\n이날 포스테코글루 감독은 경기 전 “맨유전을 준비하는 시간이 촉박했다. 손흥민이 경기에 나설 상태가 아니라고 판단, 그 없이 경기를 치르기로 했다”면서 “전력에 큰 타격이지만 핵심 선수가 빠지더라도 그에 맞춰 대처하는 방법을 배워야 한다”고 설명했다.\n손흥민이 이른 시기 복귀한다면 10월 4일 헝가리 부다페스트에서 펼쳐지는 페렌츠바로시와 UEL 2차전에 나설 전망이다.\n(서울=뉴스1)', 'source_ur

INFO:root:page_category: 스포츠
INFO:root:category: 스포츠


{'title': '홍명보호 10월 A매치 명단 발표…‘부상’ 손흥민 포함·권혁규 첫 발탁', 'time': '2024-09-30T11:22:00+09:00', 'content': '‘부상 이슈’ 손흥민 대표팀 승선\n‘유럽파 미드필더’ 권혁규 첫 발탁\n한국 남자 축구 대표팀을 이끄는 홍명보 감독이 10월 A매치에 나설 명단을 공개했다. ‘주장’ 손흥민(32·토트넘)이 부상 이슈에도 불구하고 대표팀에 합류한다.\n홍명보 감독은 30일 서울 종로구 신문로 축구회관에서 ‘2026 국제축구연맹(FIFA) 북중미월드컵’ 아시아 지역 3차 예선 3, 4차전에 출전할 26명의 국가대표 명단을 발표했다.\n지난 9월 A매치를 통해 10년 만에 대표팀 복귀를 가졌던 홍 감독의 두 번째 A매치 소집이다.\n많은 관심을 모았던 손흥민은 명단에 포함됐다.\n지난 27일(한국시각) 카라바흐(아제르바이잔)와의 ‘2024~2025시즌 유럽축구연맹(UEFA) 유로파리그(UEL)’ 경기 도중 몸에 불편함을 느껴 71분만 뛰고 교체됐던 손흥민은 30일 맨체스터 유나이티드(맨유)와 2024~2025시즌 EPL 6라운드 원정 경기(3-0 토트넘 승)까지 결장했다.\n이에 10월 A매치에서 제외될 가능성도 제기됐으나, 홍 감독은 손흥민을 이번 명단에 포함했다.\n손흥민뿐 아니라 이재성(마인츠), 황희찬(울버햄튼), 황인범(페예노르트), 이강인(파리 생제르맹), 김민재(바이에른 뮌헨) 등 주축 해외파들은 대부분 수집됐다.\n그리고 최근 스코틀랜드 하이버니언으로 이적한 뒤 좋은 모습을 보인 미드필더 권혁규(23)가 생애 처음으로 태극마크를 달았다.\n또 오현규(헹크), 백승호(버밍엄시티), 배준호(스토크시티), 김주성(FC서울), 박민규(콘사도레 삿포로) 등 소속팀에서 좋은 활약을 펼쳤던 선수들도 다시 대표팀에 소집됐다.\n한국은 10월10일 오후 11시 요르단 암만에서 요르단과 3차전 원정 경기를 치른다.\n이어 15일 오후 8시 용인미르스타디움에서 이라크와 4차전 홈 경기를 벌인다.\n이번 2